<a href="https://colab.research.google.com/github/cocogne2/projet6/blob/master/projet2_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U tensorflow-addons

In [9]:
import tensorflow as tf
import timeit
from google.colab import drive 
from lxml import etree
import os
import tensorflow_addons as tfa

drive.mount('/content/gdrive', force_remount=True)
path_to_data = os.path.join('gdrive','My Drive','projet_6')


Mounted at /content/gdrive


In [10]:
def autocontrast(image):
  """Implements Autocontrast function from PIL using TF ops.
  Args:
    image: A 3D uint8 tensor.
  Returns:
    The image after it has had autocontrast applied to it and will be of type
    uint8.
  """

  def scale_channel(image):
    """Scale the 2D image using the autocontrast rule."""
    # A possibly cheaper version can be done using cumsum/unique_with_counts
    # over the histogram values, rather than iterating over the entire image.
    # to compute mins and maxes.
    lo = tf.cast(tf.reduce_min(image), tf.float32)
    hi = tf.cast(tf.reduce_max(image), tf.float32)

    # Scale the image, making the lowest value 0 and the highest value 255.
    def scale_values(im):
      scale = 255.0 / (hi - lo)
      offset = -lo * scale
      im = tf.cast(im, tf.float32) * scale + offset
      im = tf.clip_by_value(im, 0.0, 255.0)
      return tf.cast(im, tf.uint8)

    result = tf.cond(hi > lo, lambda: scale_values(image), lambda: image)
    return result

  # Assumes RGB for now.  Scales each channel independently
  # and then stacks the result.
  s1 = scale_channel(image[:, :, 0])
  s2 = scale_channel(image[:, :, 1])
  s3 = scale_channel(image[:, :, 2])
  image = tf.stack([s1, s2, s3], 2)
  return image




In [11]:
def equalize(image):
  """Implements Equalize function from PIL using TF ops."""
  def scale_channel(im, c):
    """Scale the data in the channel to implement equalize."""
    im = tf.cast(im[:, :, c], tf.int32)
    # Compute the histogram of the image channel.
    histo = tf.histogram_fixed_width(im, [0, 255], nbins=256)

    # For the purposes of computing the step, filter out the nonzeros.
    nonzero = tf.where(tf.not_equal(histo, 0))
    nonzero_histo = tf.reshape(tf.gather(histo, nonzero), [-1])
    step = (tf.reduce_sum(nonzero_histo) - nonzero_histo[-1]) // 255

    def build_lut(histo, step):
      # Compute the cumulative sum, shifting by step // 2
      # and then normalization by step.
      lut = (tf.cumsum(histo) + (step // 2)) // step
      # Shift lut, prepending with 0.
      lut = tf.concat([[0], lut[:-1]], 0)
      # Clip the counts to be in range.  This is done
      # in the C code for image.point.
      return tf.clip_by_value(lut, 0, 255)

    # If step is zero, return the original image.  Otherwise, build
    # lut from the full histogram and step and then index from it.
    result = tf.cond(tf.equal(step, 0),
                     lambda: im,
                     lambda: tf.gather(build_lut(histo, step), im))

    return tf.cast(result, tf.uint8)

  # Assumes RGB for now.  Scales each channel independently
  # and then stacks the result.
  s1 = scale_channel(image, 0)
  s2 = scale_channel(image, 1)
  s3 = scale_channel(image, 2)
  image = tf.stack([s1, s2, s3], 2)
  return image

In [ ]:
def gpu():
  with tf.compat.v1.Session() as sess:
    with tf.device('/device:GPU:0'):
      path=os.path.join(path_to_data,'Images')
      for folder in os.listdir(path): #
          if os.path.isdir(os.path.join(path,folder)):
              print("folfer",folder)
              os.makedirs(os.path.join(path_to_data,'images_crop',folder))
              path2=os.path.join(path,folder)
              for file in os.listdir(path2):
                file=file.split(".")[0]
                print("file:",file)
                file=file.split(".")[0]
                tree = etree.parse(os.path.join(path_to_data,'Annotation', folder, file))
                for x in tree.xpath("/annotation/object/bndbox/xmin"):
                    x_min=tf.constant(int(x.text))
                for x in tree.xpath("/annotation/object/bndbox/xmax"):
                    x_max=tf.constant(int(x.text))
                for y in tree.xpath("/annotation/object/bndbox/ymin"):
                    y_min=tf.constant(int(y.text))
                for y in tree.xpath("/annotation/object/bndbox/ymax"):
                    y_max=tf.constant(int(y.text))
                #image = io.imread(os.path.join(path_to_data,'Images', folder, file+".jpg"))               
                #io.imsave(os.path.join(path_to_data,'images_crop',folder, file+".jpg"),image[y_min:y_max,x_min:x_max])
                            
                # Import the image
                image_string=tf.io.read_file(os.path.join(path_to_data,'Images', folder, file+".jpg"))
                image = tf.image.decode_jpeg(image_string, channels=0)

                #print(session.run(image))

                # Crop the image as per the parameters
                #image=autocontrast(image)
                #image=equalize(image)
                #image = tfa.image.mean_filter2d(image, filter_shape=11)
                #plt.imshow(image)
                cropped_image_tensor = tf.image.crop_to_bounding_box(image, y_min, x_min, y_max-y_min,x_max-x_min )
                output_image = tf.image.encode_jpeg(cropped_image_tensor)

                # Create a constant as filename
                file_name = tf.constant(os.path.join(path_to_data,'images_crop',folder, file+".jpg"))
                file = tf.io.write_file(file_name, output_image)

                print(sess.run(file))
                
                #print("Image Saved!")
gpu()

folfer n02085782-Japanese_spaniel
file: n02085782_1085
None
file: n02085782_1077
None
file: n02085782_1059
None
file: n02085782_1058
None
file: n02085782_1039
None
file: n02085782_1191
None
file: n02085782_1156
None
file: n02085782_1224
None
file: n02085782_1143
None
file: n02085782_1401
None
file: n02085782_1348
None
file: n02085782_1267
None
file: n02085782_1284
None
file: n02085782_1353
None
file: n02085782_1350
None
file: n02085782_1521
None
file: n02085782_1600
None
file: n02085782_1503
None
file: n02085782_1460
None
file: n02085782_1665
None
file: n02085782_1434
None
file: n02085782_17
None
file: n02085782_1610
None
file: n02085782_1552
None
file: n02085782_143
None
file: n02085782_1691
None
file: n02085782_1425
None
file: n02085782_1656
None
file: n02085782_1528
None
file: n02085782_1626
None
file: n02085782_1855
None
file: n02085782_1774
None
file: n02085782_1836
None
file: n02085782_186
None
file: n02085782_1750
None
file: n02085782_1778
None
file: n02085782_1848
None
file: n0